## Viz complémentaire axe 2.3

Silvia 16/05 - "Thierry, que en penses tu de demander une carte des destinations de navires chargés du sel (tonnage total?) sortis de Marennes pour explorer la question des marchés et des acteurs ?
avec possibilité de choisir pavillon français / pavillon étranger ?"

Nous allons essayer d'utiliser à la fois les données Toflit et les données Navigo :

- pointcalls Navigo : pointcal_action = 'Out', year = 1789, ferme_bureau = 'Marennes' (ou pointcall = 'Marennes', 'Ribérou', 'La Perrotine', 'Saint Denis d'Oléron', 'Le Chateau d' Oléron', 'Soubise', 'Saint Savinien', 'Royan', 'Anveaux'), je peux regarder ship_flag_standardized_fr pour le pavillon

**=> que puis je faire pour avoir la bonne destination ? à priori je ne peux pas utiliser les flows pour 1789**
**plan actuel : choisir pointcalls In qui ont le même pointcall Id**

- flows Toflit : year = 1789, customs_office = 'Marennes'

**(pour le pavillon je peux regarder partner_revolutionempire ?? ce n'est pas le pavillon à proprement parler)**


In [1]:
# Load an empty map
from keplergl import KeplerGl
from poitousprint import Portic, Toflit, combine_commodity_purposes, get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()
toflit_client = Toflit()

In [2]:
pointcalls_navigo = portic_client.get_pointcalls(
    year = 1789, 
    ferme_bureau = 'Marennes', # après vérification, tous les pointcalls 'Out' de 1789 qui concernent les ports rattachés au bureau des Fermes de Marennes ont bien l'attribut ferme_bureau à 'Marennes'
    pointcall_action = 'Out'
)

print("nombre de pointcalls sortant de Marennes avec du sel :", len(pointcalls_navigo))

nombre de pointcalls sortant de Marennes avec du sel : 2172


In [3]:
pointcall_test = pointcalls_navigo[0] 
# verifier ce à quoi peut ressembler un pointcall correspondant
combine_commodity_purposes(pointcall_test)

{'pkid': 77030,
 'record_id': '00333308',
 'pointcall': 'Ribérou [Saujon]',
 'pointcall_uhgs_id': 'A1964767',
 'toponyme_fr': 'Ribérou',
 'toponyme_en': 'Ribérou',
 'latitude': '45.6755',
 'longitude': '-0.935826',
 'pointcall_admiralty': 'Marennes',
 'pointcall_province': 'Saintonge',
 'pointcall_states': '[{"1749-1815" : "France"}]',
 'pointcall_substates': None,
 'pointcall_states_en': '[{"1749-1815" : "France"}]',
 'pointcall_substates_en': None,
 'state_1789_fr': 'France',
 'state_1789_en': 'France',
 'substate_1789_fr': None,
 'substate_1789_en': None,
 'nb_conges_1787_inputdone': 480.0,
 'nb_conges_1787_cr': 480.0,
 'nb_conges_1789_inputdone': 434.0,
 'nb_conges_1789_cr': 434.0,
 'pointcall_status_uncertainity': None,
 'pointcall_status': 'oblique',
 'shiparea': 'ACE-ROCH',
 'pointcall_point': '0101000020110F0000CA2ED9C7FA6EF9C0334934DF3CDA5541',
 'ferme_direction': 'La Rochelle',
 'ferme_bureau': 'Marennes',
 'ferme_bureau_uncertainty': 0.0,
 'partner_balance_1789': None,
 'par

In [4]:
pointcalls_navigo_with_revolution_empire_products = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls_navigo, product_classification="product_revolutionempire")
print(pointcalls_navigo_with_revolution_empire_products[100:110])

[{'pkid': 82184, 'record_id': '00338462', 'pointcall': 'Marennes', 'pointcall_uhgs_id': 'A0136930', 'toponyme_fr': 'Marennes', 'toponyme_en': 'Marennes', 'latitude': '45.816667', 'longitude': '-1.116667', 'pointcall_admiralty': 'Marennes', 'pointcall_province': 'Saintonge', 'pointcall_states': '[{"1749-1815" : "France"}]', 'pointcall_substates': None, 'pointcall_states_en': '[{"1749-1815" : "France"}]', 'pointcall_substates_en': None, 'state_1789_fr': 'France', 'state_1789_en': 'France', 'substate_1789_fr': None, 'substate_1789_en': None, 'nb_conges_1787_inputdone': 1172.0, 'nb_conges_1787_cr': 1172.0, 'nb_conges_1789_inputdone': 1481.0, 'nb_conges_1789_cr': 1480.0, 'pointcall_status_uncertainity': 0.0, 'pointcall_status': 'siège amirauté', 'shiparea': 'ACE-ROCH', 'pointcall_point': '0101000020110F0000CC2247D42C59FEC031FEA89F3AF05541', 'ferme_direction': 'La Rochelle', 'ferme_bureau': 'Marennes', 'ferme_bureau_uncertainty': 0.0, 'partner_balance_1789': None, 'partner_balance_supp_1789'

In [11]:
# filtrer les pointcalls pour lesquels la cargaison contient du sel
final_pointcalls_navigo = []
final_pointcalls_source_doc_ids = [] # va contenir tous les pkids des pointcalls Out dont on veut trouvé la destination => pointcall In avec le même pkid

for pointcall in pointcalls_navigo_with_revolution_empire_products:
    for commodity_purpose in pointcall['commodity_purposes']:
        if (commodity_purpose['commodity_as_toflit'] == 'Sel'):
            final_pointcalls_navigo.append(pointcall)
            final_pointcalls_source_doc_ids.append(pointcall['source_doc_id'])
        
print("Nombre de pointcalls pertinents trouvés ('Out' de Marennes en 1789 et transportant du sel) :", len(final_pointcalls_navigo))

print("\nExemple de source_doc_id récupérés : ", final_pointcalls_source_doc_ids[0:20])

print("\nExemple de pointcall pertinent : ", final_pointcalls_navigo[0])

Nombre de pointcalls pertinents trouvés ('Out' de Marennes en 1789 et transportant du sel) : 1303

Exemple de source_doc_id récupérés :  ['00337848', '00338477', '00338598', '00338453', '00338466', '00338726', '00338472', '00338562', '00338926', '00338432', '00338260', '00338273', '00338458', '00338356', '00338433', '00338437', '00338440', '00338441', '00338442', '00338444']

Exemple de pointcall pertinent :  {'pkid': 81570, 'record_id': '00337848', 'pointcall': 'Marennes', 'pointcall_uhgs_id': 'A0136930', 'toponyme_fr': 'Marennes', 'toponyme_en': 'Marennes', 'latitude': '45.816667', 'longitude': '-1.116667', 'pointcall_admiralty': 'Marennes', 'pointcall_province': 'Saintonge', 'pointcall_states': '[{"1749-1815" : "France"}]', 'pointcall_substates': None, 'pointcall_states_en': '[{"1749-1815" : "France"}]', 'pointcall_substates_en': None, 'state_1789_fr': 'France', 'state_1789_en': 'France', 'substate_1789_fr': None, 'substate_1789_en': None, 'nb_conges_1787_inputdone': 1172.0, 'nb_con

In [10]:
# retrouver les destinations
pointcalls_navigo_in = portic_client.get_pointcalls(
    start_year = 1789,
    end_year = 1790, # on considère que le voyage ne dure pas plus d'un an
    pointcall_action = ['In', 'in'] # peut être encore des soucis de casse dans l'API 
)

print("Nombre de pointcalls In trouvés entre 1789 et 1790 :", len(pointcalls_navigo_in))

print("\nExemple de pointcall pertinent : ", pointcalls_navigo_in[0])

Nombre de pointcalls In trouvés entre 1789 et 1790 : 15040

Exemple de pointcall pertinent :  {'pkid': 95234, 'record_id': '00351789', 'pointcall': 'Chalon', 'pointcall_uhgs_id': 'A1969195', 'toponyme_fr': 'Châlon', 'toponyme_en': 'Châlon', 'latitude': '45.724946', 'longitude': '-0.963790', 'pointcall_admiralty': 'Marennes', 'pointcall_province': 'Saintonge', 'pointcall_states': '[{"1749-1815" : "France"}]', 'pointcall_substates': None, 'pointcall_states_en': '[{"1749-1815" : "France"}]', 'pointcall_substates_en': None, 'state_1789_fr': 'France', 'state_1789_en': 'France', 'substate_1789_fr': None, 'substate_1789_en': None, 'nb_conges_1787_inputdone': None, 'nb_conges_1787_cr': None, 'nb_conges_1789_inputdone': None, 'nb_conges_1789_cr': None, 'pointcall_status_uncertainity': None, 'pointcall_status': None, 'shiparea': 'ACE-ROCH', 'pointcall_point': '0101000020110F000086B2E1CA8931FAC0A1CEC129EFE15541', 'ferme_direction': None, 'ferme_bureau': None, 'ferme_bureau_uncertainty': None, 'pa

In [12]:
# filtrer les pointcalls In considérés comme destination des Out sortant de Marennes avec du sel (même pkid)
final_pointcalls_in = []

for pointcall in pointcalls_navigo_in:
    if (str(pointcall['source_doc_id']) in final_pointcalls_source_doc_ids):
        final_pointcalls_in.append(pointcall)

print("Nombre de pointcalls pertinents trouvés ('In' : destination des sorties de sel de Marennes en 1789) :", len(final_pointcalls_in))

print("\nExemple de pointcall pertinent : ", final_pointcalls_in[0])

Nombre de pointcalls pertinents trouvés ('In' : destination des sorties de sel de Marennes en 1789) : 1303

Exemple de pointcall pertinent :  {'pkid': 91268, 'record_id': '00347749', 'pointcall': 'La Rochelle', 'pointcall_uhgs_id': 'A0198999', 'toponyme_fr': 'La Rochelle', 'toponyme_en': 'La Rochelle', 'latitude': '46.166667', 'longitude': '-1.15', 'pointcall_admiralty': 'La Rochelle', 'pointcall_province': 'Aunis', 'pointcall_states': '[{"1749-1815" : "France"}]', 'pointcall_substates': None, 'pointcall_states_en': '[{"1749-1815" : "France"}]', 'pointcall_substates_en': None, 'state_1789_fr': 'France', 'state_1789_en': 'France', 'substate_1789_fr': None, 'substate_1789_en': None, 'nb_conges_1787_inputdone': 1221.0, 'nb_conges_1787_cr': None, 'nb_conges_1789_inputdone': 1060.0, 'nb_conges_1789_cr': None, 'pointcall_status_uncertainity': 0.0, 'pointcall_status': 'siège amirauté', 'shiparea': 'ACE-ROCH', 'pointcall_point': '0101000020110F000054C16EA11641FFC0ACD86784FE265641', 'ferme_dire

In [19]:
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

import networkx as nx

def build_dep_dest_graph(data_dep, data_dest, key_1, key_2, **kwargs):
    """
    Cette fonction prend un ensemble de dict et deux noms de clés.
    Elle renvoie un graphe networkx de coocurrence entre les dicts
    """
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)",
        "color_2": "rgb(255, 0, 0)",
        "node_min_size": 1,
        "node_max_size": 10
    }
    params = {
        **default_params,
        **kwargs
    }

    # remplir les dicts
    for datum_dep in data_dep:
        for datum_dest in data_dest :
            if (key_1 in datum_dep and key_2 in datum_dest and datum_dep['source_doc_id'] == datum_dest['source_doc_id']):
                value_1 = datum_dep[key_1] if datum_dep[key_1] is not None else "undefined"
                value_2 = datum_dest[key_2] if datum_dest[key_2] is not None else "undefined"
                value_1_id = key_1 + "_" + value_1
                value_2_id = key_2 + "_" + value_2
                if value_1_id in key1_uniq:
                    key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
                else:
                   key1_uniq[value_1_id] = {
                       "type": key_1,
                       "name": value_1,
                       "color": params["color_1"],
                       "size": 1
                   }
                if value_2_id in key2_uniq:
                    key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
                else:
                   key2_uniq[value_2_id] = {
                       "type": key_2,
                       "name": value_2,
                       "color": params["color_2"],
                       "size": 1
                   }
                edge_footprint = value_1_id + "-" + value_2_id
                if edge_footprint in edges_uniq:
                    edges_uniq[edge_footprint]["weight"] += 1
                else:
                    edges_uniq[edge_footprint] = {
                        "source": value_1_id,
                        "target": value_2_id,
                        "weight": 1
                    }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))

    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [params["node_min_size"], params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Graph

In [20]:
from ipysigma import Sigma

Graph = build_dep_dest_graph(final_pointcalls_navigo, final_pointcalls_in, 'pointcall', 'pointcall')
Sigma(Graph, start_layout=True)

Sigma(data={'nodes': [('pointcall_Marennes', {'type': 'pointcall', 'name': 'Marennes', 'color': 'rgb(0, 255, 0…

In [ ]:
# tentative d'obtenir un nest_portic_pointcall qui fonctionne (même s'il oublie certains arguments)
fieldnames = portic_client.get_fieldnames()

fieldnames[0:5]

In [ ]:
fieldnames_list = []
for field in fieldnames:
    fieldnames_list.append(field['name'])
    
fieldnames_list